In [ ]:
import kwant
from math import sin, cos
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def NOT(shape):
    """returns not(shape)"""
    def _shape(site):
        return not shape(site)
    return _shape

def UNION(*shapes):
    """returns shape which is an OR function applied to shapes"""
    def _shape(site):
        res = False
        for shape in shapes:
            res |= shape(site)
        return res
    return _shape
    
def INTERSECTION(*shapes):
    """returns shape which is an AND function applied to shapes"""
    def _shape(site):
        res = True
        for shape in shapes:
            res &= shape(site)
        return res
    return _shape
    
def DIFFERENCE(shape_A, shape_B):
    """returns shape which is true when the site is in the first shape, but not in the second"""
    def _shape(site):
        return shape_A(site) and not shape_B(site)
    return _shape

def EDGE(shape): 
    def _shape(site):
        x, y = site.pos
        neighboring_sites = {}
        neighboring_sites['wsite']  = kwant.builder.Site(site.family, site.tag + [-1, 0])
        neighboring_sites['esite']  = kwant.builder.Site(site.family, site.tag + [+1, 0])
        neighboring_sites['nsite']  = kwant.builder.Site(site.family, site.tag + [ 0,+1])
        neighboring_sites['ssite']  = kwant.builder.Site(site.family, site.tag + [ 0,-1])
        neighboring_sites['nwsite'] = kwant.builder.Site(site.family, site.tag + [-1,+1])
        neighboring_sites['nesite'] = kwant.builder.Site(site.family, site.tag + [+1,+1])
        neighboring_sites['swsite'] = kwant.builder.Site(site.family, site.tag + [-1,-1])
        neighboring_sites['sesite'] = kwant.builder.Site(site.family, site.tag + [+1,-1])
        return shape(site) and not all([shape(neighbouring_site) for neighbouring_site in neighboring_sites.values()])
    return _shape

def INTERIOR(shape):
    return DIFFERENCE(shape, EDGE(shape))

def below_curve(curve):
    def _shape(site):
        x, y = site.pos
        return y < curve(x)
    return _shape

def above_curve(curve):
    return NOT(below_curve(curve))

def left_of_curve(curve):
    def _shape(site):
        x, y = site.pos
        return x < curve(y)
    return _shape

def right_of_curve(curve):
    return NOT(left_of_curve(curve))

In [ ]:
def add_to_site_colors(site_colors, marked_sites, color):
    site_colors.update({site: color for site in marked_sites})
    
def site_color_function(site_color, syst):
    return [site_color[site] for site in syst.sites]


# Make simple builder

In [ ]:
template_interior = kwant.continuum.discretize('k_x^2 + k_y^2', grid_spacing=10)
template_edge = kwant.continuum.discretize('k_x^2 + k_y^2 + 1000', grid_spacing=10)
template_top_superconductor = kwant.continuum.discretize('k_x^2 + k_y^2 + 100', grid_spacing=10)
template_bottom_superconductor = kwant.continuum.discretize('k_x^2 + k_y^2 - 100', grid_spacing=10)

syst = kwant.Builder()

#######################
# Define middle section
curve = lambda x: 100 + 50*sin(6 / 800 * x)
below_shape = below_curve(curve)
above_shape = above_curve(lambda x: curve(x) - 200)
right_boundary = left_of_curve(lambda y: 800)
left_boundary = right_of_curve(lambda y: 0)
middle_shape = INTERSECTION(below_shape, above_shape, right_boundary, left_boundary)


#############
# Define edge
edge_shape = EDGE(middle_shape)
edge_initial_site = (0,0)

#########################
# Remove edge from middle
interior_shape = INTERIOR(middle_shape)
interior_initial_site = (20,20)

############################
# Define top superconductor
top_superconductor_border = lambda x: 400
top_superconductor_shape = INTERSECTION(DIFFERENCE(below_curve(top_superconductor_border), middle_shape),
                                        left_boundary, right_boundary)
top_superconductor_initial_site = (0,100)

##############################
# Define bottom superconductor
bottom_superconductor_border = lambda x: -400
bottom_superconductor_shape = INTERSECTION(DIFFERENCE(above_curve(bottom_superconductor_border), middle_shape),
                                        left_boundary, right_boundary)
bottom_superconductor_initial_site = (0,-110)

############
# Fill sites
site_colors = dict()

    # Fill edge
edge_sites = syst.fill(template_edge, edge_shape, edge_initial_site)
add_to_site_colors(site_colors, edge_sites, 'green')

    # Fill interior
interior_sites = syst.fill(template_interior, interior_shape, interior_initial_site)
add_to_site_colors(site_colors, interior_sites, 'black')

    # Fill top superconductor
top_superconductor_sites = syst.fill(template_top_superconductor, top_superconductor_shape, top_superconductor_initial_site)
add_to_site_colors(site_colors, top_superconductor_sites, 'gold')

    # Fill bottom superconductor
bottom_superconductor_sites = syst.fill(template_bottom_superconductor, bottom_superconductor_shape, bottom_superconductor_initial_site)
add_to_site_colors(site_colors, bottom_superconductor_sites, 'gold')



#################
# Finalize system
syst = syst.finalized()

In [ ]:
kwant.plot(syst, site_color=site_color_function(site_colors, syst), dpi=200);